# Importing Libraries

In [1]:
#!pip install torch-spline-conv
#!pip install torch-geometric-temporal --no-dependencies

In [2]:
import torchvision
import torchaudio
import pandas
import gensim
import torch_scatter
import torch_sparse
import torch_cluster
import setuptools.dist
#import torch-spline-conv
import torch_geometric
import networkx
import matplotlib
import node2vec
import seaborn
import sklearn
import tensorflow
import deepchem
#import torch_geometric_temporal
import captum

/Users/ishaanpaul/Desktop/Extras/Graph-Neural-Network/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


# Chapter 2

In [3]:
import networkx as nx

In [4]:
# To create a unweighted, undirected graph
G = nx.Graph()
G.add_edges_from([('A', 'B'), ('A', 'C'), ('B', 'D'),
('B', 'E'), ('C', 'F'), ('C', 'G')])

In [5]:
# To create a directed graph
DG = nx.DiGraph()
DG.add_edges_from([('A', 'B'), ('A', 'C'), ('B', 'D'),
('B', 'E'), ('C', 'F'), ('C', 'G')])

In [6]:
# To create a weighted graph
WG = nx.Graph()
WG.add_edges_from([('A', 'B', {"weight": 10}), ('A', 'C',
{"weight": 20}), ('B', 'D', {"weight": 30}), ('B', 'E',
{"weight": 40}), ('C', 'F', {"weight": 50}), ('C', 'G',
{"weight": 60})])
labels = nx.get_edge_attributes(WG, "weight")

In [7]:
# To create connected graphs
G1 = nx.Graph()
G1.add_edges_from([(1, 2), (2, 3), (3, 1), (4, 5)])
# This should come back as false since 1,2,3 are connected and 4,5 is disconnected
print(f"Is graph 1 connected? {nx.is_connected(G1)}")
G2 = nx.Graph()
G2.add_edges_from([(1, 2), (2, 3), (3, 1), (1, 4)])
# This should come back as true since there is a path between any nodes u, v in graph G2
print(f"Is graph 2 connected? {nx.is_connected(G2)}")

Is graph 1 connected? False
Is graph 2 connected? True


In [8]:
# Calculating the degree of a node in an undirected graph
G = nx.Graph()
G.add_edges_from([('A', 'B'), ('A', 'C'), ('B', 'D'), ('B',
'E'), ('C', 'F'), ('C', 'G')])
print(f"deg(A) = {G.degree['A']}")

deg(A) = 2


In [9]:
# Calculating the degree of a node in a directed graph
DG = nx.DiGraph()
DG.add_edges_from([('A', 'B'), ('A', 'C'), ('B', 'D'), ('B',
'E'), ('C', 'F'), ('C', 'G')])
print(f"deg^-(A) = {DG.in_degree['A']}")
print(f"deg^+(A) = {DG.out_degree['A']}")

deg^-(A) = 0
deg^+(A) = 2


In [10]:
# Finding the following measures related to a graph
# 1. Degree Centrality - Degree of a node. The higher the degree is, the more influential the node is
# 2. Closeness Centrality - Average length of the shortest path between the target node and all other nodes. Node with high closeness centrality can reach other nodes very easily
# 3. Betweenness Centrality - How often a node appears in the shortest path of 2 other nodes. It identifies bottlenecks or bridges
print(f"Degree centrality = {nx.degree_centrality(G)}")
print(f"Closeness centrality = {nx.closeness_centrality(G)}")
print(f"Betweenness centrality = {nx.betweenness_centrality(G)}")

Degree centrality = {'A': 0.3333333333333333, 'B': 0.5, 'C': 0.5, 'D': 0.16666666666666666, 'E': 0.16666666666666666, 'F': 0.16666666666666666, 'G': 0.16666666666666666}
Closeness centrality = {'A': 0.6, 'B': 0.5454545454545454, 'C': 0.5454545454545454, 'D': 0.375, 'E': 0.375, 'F': 0.375, 'G': 0.375}
Betweenness centrality = {'A': 0.6, 'B': 0.6, 'C': 0.6, 'D': 0.0, 'E': 0.0, 'F': 0.0, 'G': 0.0}


In [11]:
# Creating an adjacency matrix
# This is great for matrix operations as well as checking the connectivity of two nodes in constant time
# Its space complexity is O(|V|^2) and is therefore difficult to resize and add new nodes, especially for larger graphs
adj = [[0,1,1,0,0,0,0],
[1,0,0,1,1,0,0],
[1,0,0,0,0,1,1],
[0,1,0,0,0,0,0],
[0,1,0,0,0,0,0],
[0,0,1,0,0,0,0],
[0,0,1,0,0,0,0]]

In [12]:
# An alternative to an adjacency matrix
# Space complexity O(|E|)
# Retrieving connectivity of two nodes requires traversing the list and therefore is more useful for situations where space is a concern
edge_list = [(0, 1), (0, 2), (1, 3), (1, 4), (2, 5), (2, 6)]

In [13]:
# Another way of representation is the adjacency list
# Space complexity: O(|V|+|E|)
# It allows for efficient iteration through the adjacent vertices of a node
# checking whether two vertices are connected can be slower than with an adjacency matrix.
adj_list = {
0: [1, 2],
1: [0, 3, 4],
2: [0, 5, 6],
3: [1],
4: [1],
5: [2],
6: [2]
}

Breadth-First Search

BFS is particularly useful in finding the shortest path between two nodes in an unweighted graph
In addition to finding the shortest path, BFS can also be used to check whether a graph is connected or to find all connected components of a graph.
The time complexity of BFS is O(|V| + |E|)

In [14]:
G = nx.Graph()
G.add_edges_from([('A', 'B'), ('A', 'C'), ('B', 'D'),
('B', 'E'), ('C', 'F'), ('C', 'G')])

In [15]:
def bfs(graph, node):
    visited, queue = [node], [node]
    while queue:
        node = queue.pop(0)
        for neighbor in graph[node]:
            if neighbor not in visited:
                visited.append(neighbor)
                queue.append(neighbor)
    return visited

In [16]:
bfs(G, 'A')

['A', 'B', 'C', 'D', 'E', 'F', 'G']

Depth-First Search

DFS is useful in solving various problems, such as finding connected components, topological sorting, and solving maze problems. It is particularly useful in finding cycles in a graph since it traverses the graph in a depth-first order, and a cycle exists if, and only if, a node is visited twice during the traversal. The time complexity is the same as BFS i.e. O(|V| + |E|)

In [17]:
visited = []
def dfs(visited, graph, node):
    if node not in visited:
        visited.append(node)
        for neighbor in graph[node]:
            visited = dfs(visited, graph, neighbor)
    return visited

In [18]:
dfs(visited, G, 'A')

['A', 'B', 'D', 'E', 'C', 'F', 'G']